# NASA Defects - Import & Clean

## Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display, Markdown
plt.style.use("seaborn-darkgrid")
pd.set_option('display.max_columns', None)  

import sys, os, yaml

DATASET = "NASA"

COLAB = 'google.colab' in sys.modules
if COLAB:
    ROOT = f"/content/gdrive/MyDrive/datasets/{DATASET.replace(' ','_')}/"
else:
    ROOT = "./"

DEBUG = False
SEED = 1612

In [2]:
if COLAB:
    from google.colab import drive
    if not os.path.isdir("/content/gdrive"):
        drive.mount("/content/gdrive")
        d = "/content/gdrive/MyDrive/datasets"
        if not os.path.isdir(d): os.makedirs(d)
        if not os.path.isdir(ROOT): os.makedirs(ROOT)

def makedirs(d):
    if COLAB:
        if not os.path.isdir(ROOT+d): os.makedirs(ROOT+d)
    else:
        if not os.path.isdir(ROOT+d): os.makedirs(ROOT+d, mode=0o777, exist_ok=True)

for d in ['orig','data','output']: makedirs(d)

## Dataset

In [3]:
filename = "pc2.csv"
target = f"{ROOT}/orig/{filename}"

if not os.path.isfile(target):
    print (f"Downloading remote file {filename}", sep="")
    
    import urllib.request
    urllib.request.urlretrieve("https://setu-datamining2.github.io/live/topics/21-Assignments/01-NASA_Software_Defect_Datasets/files/pc2.csv", target)
    
    #df = pd.read_csv("https://setu-datamining2.github.io/live/topics/21-Assignments/01-NASA_Software_Defect_Datasets/files/pc2.csv")
    
else:
    df = pd.read_csv(target)
    print(f"Using local copy of {filename}")        


Using local copy of pc2.csv


In [4]:
df = pd.read_csv(f"orig/pc2.csv")
print(df.shape)
df.head()

(5589, 37)


,BRANCH_COUNT,CALL_PAIRS,LOC_CODE_AND_COMMENT,LOC_COMMENTS,CONDITION_COUNT,CYCLOMATIC_COMPLEXITY,CYCLOMATIC_DENSITY,DECISION_COUNT,DECISION_DENSITY,DESIGN_COMPLEXITY,DESIGN_DENSITY,EDGE_COUNT,ESSENTIAL_COMPLEXITY,ESSENTIAL_DENSITY,LOC_EXECUTABLE,PARAMETER_COUNT,HALSTEAD_CONTENT,HALSTEAD_DIFFICULTY,HALSTEAD_EFFORT,HALSTEAD_ERROR_EST,HALSTEAD_LENGTH,HALSTEAD_LEVEL,HALSTEAD_PROG_TIME,HALSTEAD_VOLUME,MAINTENANCE_SEVERITY,MODIFIED_CONDITION_COUNT,MULTIPLE_CONDITION_COUNT,NODE_COUNT,NORMALIZED_CYLOMATIC_COMPLEXITY,NUM_OPERANDS,NUM_OPERATORS,NUM_UNIQUE_OPERANDS,NUM_UNIQUE_OPERATORS,NUMBER_OF_LINES,PERCENT_COMMENTS,LOC_TOTAL,defects
0,1.0,0.0,0.0,0.0,0.0,1.0,1.00,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,2.0,5.33,1.50,12.00,0.00,4.0,0.67,0.67,8.00,1.0,0.0,0.0,2.0,0.50,1.0,3.0,1.0,3.0,2.0,0.00,0.0,False
1,1.0,1.0,0.0,0.0,0.0,1.0,1.00,0.0,0.0,1.0,1.0,2.0,1.0,0.0,0.0,1.0,0.00,0.00,0.00,0.00,1.0,0.00,0.00,0.00,1.0,0.0,0.0,3.0,1.00,1.0,0.0,1.0,0.0,1.0,0.00,0.0,False
2,1.0,4.0,7.0,24.0,0.0,1.0,0.13,0.0,0.0,1.0,1.0,6.0,1.0,0.0,1.0,0.0,17.88,7.43,986.77,0.04,34.0,0.13,54.82,132.83,1.0,0.0,0.0,7.0,0.03,13.0,21.0,7.0,8.0,34.0,96.88,8.0,False
3,1.0,1.0,11.0,3.0,0.0,1.0,0.08,0.0,0.0,1.0,1.0,2.0,1.0,0.0,1.0,0.0,42.62,7.81,2598.31,0.11,77.0,0.13,144.35,332.79,1.0,0.0,0.0,3.0,0.06,29.0,48.0,13.0,7.0,17.0,93.33,12.0,False
4,1.0,1.0,0.0,0.0,0.0,1.0,1.00,0.0,0.0,1.0,1.0,2.0,1.0,0.0,1.0,3.0,33.44,0.63,13.06,0.01,9.0,1.60,0.73,20.90,1.0,0.0,0.0,3.0,0.33,5.0,4.0,4.0,1.0,3.0,0.00,1.0,False


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5589 entries, 0 to 5588
Data columns (total 37 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   BRANCH_COUNT                     5589 non-null   float64
 1   CALL_PAIRS                       5589 non-null   float64
 2   LOC_CODE_AND_COMMENT             5589 non-null   float64
 3   LOC_COMMENTS                     5589 non-null   float64
 4   CONDITION_COUNT                  5589 non-null   float64
 5   CYCLOMATIC_COMPLEXITY            5589 non-null   float64
 6   CYCLOMATIC_DENSITY               5589 non-null   float64
 7   DECISION_COUNT                   5589 non-null   float64
 8   DECISION_DENSITY                 5589 non-null   float64
 9   DESIGN_COMPLEXITY                5589 non-null   float64
 10  DESIGN_DENSITY                   5589 non-null   float64
 11  EDGE_COUNT                       5589 non-null   float64
 12  ESSENTIAL_COMPLEXITY

In [6]:

print("Number of cases : ", df.shape[0])
print("Number of features : ", df.shape[1])


Number of cases :  5589
Number of features :  37


## Summary statistics 
 - mathcing data on table 1
 
    -number of cases = 5589
    -number of features = 37
    
- all statistics are in agree,emt with paper

In [7]:
#table_case_count = lambda df: df.shape[0]
#table_feature_count = lambda df: df.shape[1]


#messages = []
#for m, expected, observed in [
#    "Number of cases:", 5589, table_case_count,
#    "Number of features:", 37, table_feature_count,
#    ]:
#    result = "PASS" if expected==observed else "FAIL"
#    message.append(f" * {m} {expected=} {observed=} {result}")
#    
#display(Markdown("\n".join(messages)))

## Table 1 - Comparison of versions

In [8]:
table_case_count =  df.shape[0]
table_feature_count =  df.shape[1]

messages = []
for m, expected, observed in [
    ("Number of cases:", 5589, table_case_count),
    ("Number of features:", 37, table_feature_count),
    ]:
    result = "PASS" if expected==observed else "FAIL"
    messages.append([m, expected , observed, result])
                   
df_results = pd.DataFrame(messages, columns = ["Description", "Expected", "Observed", "Result"])
df_results

,Description,Expected,Observed,Result
0,Number of cases:,5589,5589,PASS
1,Number of features:,37,37,PASS


## Table 2 - Issues by features

### Identical Features

In [9]:
table_2_A = lambda df : df.T.duplicated().sum()

In [10]:
messages = []
for m, expected, observed in [
    ("Identical Features:", 0, table_2_A(df)),
]:
    result = "PASS" if expected==observed else "FAIL"
    messages.append([m,expected ,observed])
    
df_results = pd.DataFrame(messages, columns = ["Desription", "Expected", "Observed"])    
df_results["Result"] = df_results.Expected==df_results.Observed
df_results   

,Desription,Expected,Observed,Result
0,Identical Features:,0,0,True


### Constant Features 

In [11]:
table_2_B = lambda df : df.nunique()[df.nunique() == 1].shape[0]

In [12]:
messages = []
for m, expected, observed in [
    ("Constant Features:", 0, table_2_B(df)),
]:
    result = "PASS" if expected==observed else "FAIL"
    messages.append([m,expected ,observed])
    
df_results = pd.DataFrame(messages, columns = ["Desription", "Expected", "Observed"])    
df_results["Result"] = df_results.Expected==df_results.Observed
df_results

,Desription,Expected,Observed,Result
0,Constant Features:,0,0,True


### Features with missing values

In [13]:
table_2_C =  lambda df: df.isna().sum().sum()

In [14]:
messages = []
for m, expected, observed in [
    ("Features with missing values:", 0, table_2_C(df)),
]:
    result = "PASS" if expected==observed else "FAIL"
    messages.append([m,expected ,observed])
    
df_results = pd.DataFrame(messages, columns = ["Desription", "Expected", "Observed"])    
df_results["Result"] = df_results.Expected==df_results.Observed
df_results

,Desription,Expected,Observed,Result
0,Features with missing values:,0,0,True


### Features with conflicting values

In [15]:
check1 = df.query('NUMBER_OF_LINES < LOC_TOTAL')
#check2 = df.query('NUMBER_OF_LINES >= LOC_BLANK')
check3 = df.query('NUMBER_OF_LINES < LOC_CODE_AND_COMMENT')
check4 = df.query('NUMBER_OF_LINES < LOC_COMMENTS')
check5 = df.query('NUMBER_OF_LINES < LOC_EXECUTABLE')
check6 = df.query('LOC_TOTAL < LOC_EXECUTABLE')
check7 = df.query('LOC_TOTAL < LOC_CODE_AND_COMMENT')
check8 = df.query('NUM_OPERANDS < NUM_UNIQUE_OPERANDS')
check9 = df.query('NUM_OPERATORS < NUM_UNIQUE_OPERATORS')
check10 = df.query('HALSTEAD_LENGTH != NUM_OPERATORS + NUM_OPERANDS')
check11 = df.query('CYCLOMATIC_COMPLEXITY > NUM_OPERATORS + 1')
check12 = df.query('CALL_PAIRS > NUM_OPERATORS')
#check13 = df.query('HALSTEAD_VOLUME != (NUM_OPERATORS + NUM_OPERANDS) * (np.log2(NUM_UNIQUE_OPERATORS + NUM_UNIQUE_OPERANDS))')
#check14 = df.query('HALSTEAD_LEVEL != (2/NUM_UNIQUE_OPERATORS) * (NUM_UNIQUE_OPERANDS/NUM_OPERANDS)')
#check15 = df.query('HALSTEAD_DIFFICULTY != (NUM_UNIQUE_OPERATORS/2) * (NUM_OPERANDS/NUM_UNIQUE_OPERANDS)')
#check16 = df.query('HALSTEAD_CONTENT != HALSTEAD_VOLUME/HALSTEAD_DIFFICULTY')
#check17 = df.query('HALSTEAD_EFFORT != HALSTEAD_VOLUME * HALSTEAD_DIFFICULTY')
#check18 = df.query('HALSTEAD_PROG_TIME != HALSTEAD_EFFORT/18')

In [16]:
columns_with_fails = []

# Check condition in check1
if check1.shape[0] > 1:
    columns_with_fails.extend(['NUMBER_OF_LINES', 'LOC_TOTAL'])

# Check condition in check3
if check3.shape[0] > 1:
    columns_with_fails.extend(['NUMBER_OF_LINES', 'LOC_CODE_AND_COMMENT'])

# Check condition in check4
if check4.shape[0] > 1:
    columns_with_fails.extend(['NUMBER_OF_LINES', 'LOC_COMMENTS'])

# Check condition in check5
if check5.shape[0] > 1:
    columns_with_fails.extend(['NUMBER_OF_LINES', 'LOC_EXECUTABLE'])

# Check condition in check6
if check6.shape[0] > 1:
    columns_with_fails.extend(['LOC_TOTAL', 'LOC_EXECUTABLE'])

# Check condition in check7
if check7.shape[0] > 1:
    columns_with_fails.extend(['LOC_TOTAL', 'LOC_CODE_AND_COMMENT'])

# Check condition in check8
if check8.shape[0] > 1:
    columns_with_fails.extend(['NUM_OPERANDS', 'NUM_UNIQUE_OPERANDS'])

# Check condition in check9
if check9.shape[0] > 1:
    columns_with_fails.extend(['NUM_OPERATORS', 'NUM_UNIQUE_OPERATORS'])

# Check condition in check10
if check10.shape[0] > 1:
    columns_with_fails.extend(['HALSTEAD_LENGTH', 'NUM_OPERATORS', 'NUM_OPERANDS'])

# Check condition in check11
if check11.shape[0] > 1:
    columns_with_fails.extend(['CYCLOMATIC_COMPLEXITY', 'NUM_OPERATORS'])

# Check condition in check12
if check12.shape[0] > 1:
    columns_with_fails.extend(['CALL_PAIRS', 'NUM_OPERATORS'])

columns_with_fails = list(set(columns_with_fails))

print(columns_with_fails)

['CALL_PAIRS', 'NUM_OPERATORS']


In [17]:
table_2_D=len(columns_with_fails)

messages = []
for m, expected, observed in [
    ("Features with conflicting values:", 2, table_2_D),
]:
    result = "PASS" if expected==observed else "FAIL"
    messages.append([m,expected ,observed])
    
df_results = pd.DataFrame(messages, columns = ["Desription", "Expected", "Observed"])    
df_results["Result"] = df_results.Expected==df_results.Observed
df_results

,Desription,Expected,Observed,Result
0,Features with conflicting values:,2,2,True


#### Comments
 - Integrity check 2 fails as dataframe doesnt have LOC_BLANK column therefore check 2 was ignored for this run
 - All checks from check12-check18 show that they have conflicticting values
 - this means that there are possibly 11 conflicting features
 - this does not match the expected value of 2
 
 Note: While progressing through the verification of the data to be cleaned it was discovered that integrity checks 13-18 were likely ommited so to relfect this the checks were commented out to compare the expected and compared values more accurately.

### Features with implausible values

In [18]:
def count_implausible_columns(df):
    total_implausible = 0
    implausible_columns = []
    
    for col in df.columns:
        if col not in implausible_columns:
            if ((col =='LOC_TOTAL' and (df[col]== 0).any()) or
                ((df[col] < 0).any()) or 
                ('count' in col and df[col].dtype != int)):
                    implausible_columns.append(col)
                    total_implausible += 1
            
    return total_implausible,implausible_columns

In [19]:
total_implausible,implausible_columns = count_implausible_columns(df)

messages = []
for m, expected, observed in [
    ("Features with implausible values:", 1, total_implausible),
]:
    result = "PASS" if expected==observed else "FAIL"
    messages.append([m,expected ,observed])
    
df_results = pd.DataFrame(messages, columns = ["Desription", "Expected", "Observed"])    
df_results["Result"] = df_results.Expected==df_results.Observed
print(df_results)

implausible_columns

                          Desription  Expected  Observed  Result
0  Features with implausible values:         1         1    True


['LOC_TOTAL']

### Total feature problems

In [20]:
features_before_drop = df.shape[1] 

df_total_features= df.copy()


# Drop features that fail a check
for c in columns_with_fails:
    df_total_features=df_total_features.drop(columns=c)

# Drop features that have implausible values
for col in implausible_columns :
    if col not in columns_with_fails:
        df_total_features=df_total_features.drop(columns=col)

total_problem_features = features_before_drop - df_total_features.shape[1]
total_problem_features

3

In [21]:
messages = []
for m, expected, observed in [
    ("Total features with issues:", 3, total_problem_features),
]:
    result = "PASS" if expected==observed else "FAIL"
    messages.append([m,expected ,observed])
    
df_results = pd.DataFrame(messages, columns = ["Desription", "Expected", "Observed"])    
df_results["Result"] = df_results.Expected==df_results.Observed
print(df_results)

                    Desription  Expected  Observed  Result
0  Total features with issues:         3         3    True


#### Comment
- The only feature issues found were with integrity checks and implausiblie values
- Integrity checks:
    - Since checks 12-19 all failed this left us with a list of  11 features with problems [CALL_PAIRS,NUM_OPERATORS,HALSTEAD_LEVEL,NUM_UNIQUE_OPERATORS,NUM_UNIQUE_OPERANDS,NUM_OPERANDS,HALSTEAD_DIFFICULTY,HALSTEAD_CONTENT,HALSTEAD_VOLUME,HALSTEAD_EFFORT,HALSTEAD_PROG_TIME]
- Implausible values:
    - The only column to fail any of the implausible value checks was LOC_TOTAL
    - This means we have a total of 12 feautures with issues
    
Note: This number decreased to 3 after I decided to use the same apporoach as the tables where only hte first twelve integrity checks were ran which is described in the summmary section below.

## Table 2 - Issues by cases

### Identical cases

In [22]:
table_2_G = sum(df.duplicated(keep=False))
table_2_G

4621

In [23]:
messages = []
for m, expected, observed in [
    ("Identical cases:", 4621, table_2_G),
]:
    result = "PASS" if expected==observed else "FAIL"
    messages.append([m,expected ,observed])
    
df_results = pd.DataFrame(messages, columns = ["Desription", "Expected", "Observed"])    
df_results["Result"] = df_results.Expected==df_results.Observed
df_results

,Desription,Expected,Observed,Result
0,Identical cases:,4621,4621,True


#### Comment 
 - have to use following setting to get accurate count Keep=False  as it marks all duplicates as True not just the reoccuring

### Incosistent cases

In [24]:
grouped = df.groupby(df.columns[:-1].tolist())
table_2_H = grouped.filter(lambda x: len(x['defects'].unique()) > 1)

In [25]:
messages = []
for m, expected, observed in [
    ("inconsistent  case values:", 100, table_2_H.shape[0]),
]:
    result = "PASS" if expected==observed else "FAIL"
    messages.append([m,expected ,observed])
    
df_results = pd.DataFrame(messages, columns = ["Desription", "Expected", "Observed"])    
df_results["Result"] = df_results.Expected==df_results.Observed
df_results

,Desription,Expected,Observed,Result
0,inconsistent case values:,100,100,True


### Cases with missing values

In [26]:
table_2_I = df.isnull().any(axis=1).sum()

In [27]:
messages = []
for m, expected, observed in [
    ("Cases with missing values:", 0, table_2_I),
]:
    result = "PASS" if expected==observed else "FAIL"
    messages.append([m,expected ,observed])
    
df_results = pd.DataFrame(messages, columns = ["Desription", "Expected", "Observed"])    
df_results["Result"] = df_results.Expected==df_results.Observed
df_results

,Desription,Expected,Observed,Result
0,Cases with missing values:,0,0,True


### Cases with conflicting feature value
  - reusing the checks from conflicting features

In [28]:
messages = []
for m, observed in [
    ("Conflicting feature values in check1:", check1.shape[0]),
     ("Conflicting feature values in check3:", check3.shape[0]),
     ("Conflicting feature values in  check4:", check4.shape[0]),
     ("Conflicting feature values in  check5:", check5.shape[0]),
     ("Conflicting feature values in  check6:", check6.shape[0]),
     ("Conflicting feature values in  check7:", check7.shape[0]),
     ("Conflicting feature values in  check8:", check8.shape[0]),
     ("Conflicting feature values in  check9:", check9.shape[0]),
     ("Conflicting feature values in  check10:", check10.shape[0]),
     ("Conflicting feature values in  check11:", check11.shape[0]),
     ("Conflicting feature values in  check12:", check12.shape[0]),
     #("Conflicting feature values in  check14:", check14.shape[0]),
     #("Conflicting feature values in  check15:", check15.shape[0]),
     #("Conflicting feature values in  check16:", check16.shape[0]),
     #("Conflicting feature values in  check17:", check17.shape[0]),
     #("Conflicting feature values in  check18:", check18.shape[0]),
    
]:
       messages.append([m ,observed])
    
conflict_feat_values_results = pd.DataFrame(messages, columns = ["Desription", "Observed"])    
conflict_feat_values_results

,Desription,Observed
0,Conflicting feature values in check1:,0
1,Conflicting feature values in check3:,0
2,Conflicting feature values in check4:,0
3,Conflicting feature values in check5:,0
4,Conflicting feature values in check6:,0
5,Conflicting feature values in check7:,0
6,Conflicting feature values in check8:,0
7,Conflicting feature values in check9:,0
8,Conflicting feature values in check10:,0
9,Conflicting feature values in check11:,0


In [29]:
table_2_J= sum([check1.shape[0],check3.shape[0],check4.shape[0],check5.shape[0],check6.shape[0],check7.shape[0],
                check8.shape[0],check9.shape[0],check10.shape[0],check11.shape[0],check12.shape[0]])

messages = []
for m, expected, observed in [
    ("Cases with conflicting values:", 129, table_2_J),
]:
    result = "PASS" if expected==observed else "FAIL"
    messages.append([m,expected ,observed])
    
df_results = pd.DataFrame(messages, columns = ["Desription", "Expected", "Observed"])    
df_results["Result"] = df_results.Expected==df_results.Observed
df_results

,Desription,Expected,Observed,Result
0,Cases with conflicting values:,129,129,True


#### Comment
 - The number of conflicting cases is greather than expected 
 - However if we just run the integrity checks from 1-12 then it gives us the expected value
 - This could mean that when the integrity checks were being checked only the first twelve were ran

### Cases with implausible values

In [30]:
def count_implausible_cases(df):
    total_implausible_cases = 0
    
    for col in df.columns:
            if (col =='LOC_TOTAL'):
                check_loc_total =  df.query(f'{col} == 0')
                total_implausible_cases += check_loc_total.shape[0]
            if ('count' in col):
                check_count = df.query(f'{col}.dtype != int')
                total_implausible_cases += check_count.shape[0]
            else:
                check_negative = df.query(f'{col} < 0')
                total_implausible_cases += check_negative.shape[0]

            
    return total_implausible_cases

In [31]:
table_2_K= count_implausible_cases(df)

messages = []
for m, expected, observed in [
    ("Implausible cases :", 1084, table_2_K),
]:
    result = "PASS" if expected==observed else "FAIL"
    messages.append([m,expected ,observed])
    
df_results = pd.DataFrame(messages, columns = ["Desription", "Expected", "Observed"])    
df_results["Result"] = df_results.Expected==df_results.Observed
df_results

,Desription,Expected,Observed,Result
0,Implausible cases :,1084,1084,True


### Total problem cases DS'

In [32]:
cases_before_drop = df.shape[0] 

df_DS1= df.copy()

# Drop rows that contain na values
df_DS1=df_DS1.dropna()

# Drop rows that fail any checks
failed_rows = pd.concat([check1, check3, check4, check5, check6, check7, check8, check9, check10, check11, check12])
df_DS1=df_DS1.drop(failed_rows.index)

# Drop rows that contain implausible values
df_DS1 = df_DS1.drop(df_DS1[df_DS1['LOC_TOTAL'] == 0].index)

df_DS1 = df_DS1.drop(df_DS1[(df_DS1 < 0).any(axis=1)].index)

count_columns = [col for col in df_DS1.columns if 'count' in col]
df_DS1 = df_DS1.drop(df_DS1[~df_DS1[count_columns].applymap(lambda x: isinstance(x, int)).all(axis=1)].index)

total_data_quality_problem_cases = cases_before_drop - df_DS1.shape[0]
total_data_quality_problem_cases


1163

In [33]:
messages = []
for m, expected, observed in [
    ("Problem cases I-K:", 1163, total_data_quality_problem_cases),
]:
    result = "PASS" if expected==observed else "FAIL"
    messages.append([m,expected ,observed])
    
df_results = pd.DataFrame(messages, columns = ["Desription", "Expected", "Observed"])    
df_results["Result"] = df_results.Expected==df_results.Observed
df_results

,Desription,Expected,Observed,Result
0,Problem cases I-K:,1163,1163,True


### Comment
   - the above result was obtained by only running checks1-12 as it was found earlier that likely how the results were obtained

### Total problem DS''

In [34]:
# Drop rows that contain duplicates
df_DS1=df_DS1.drop_duplicates(keep = False)

# Drop rows that contain Inconsitent values
grouped = df_DS1.groupby(df_DS1.columns[:-1].tolist())
inconsistent = grouped.filter(lambda x: len(x['defects'].unique()) > 1)
df_DS1=df_DS1.drop(inconsistent.index)

total_problem_cases = cases_before_drop - df_DS1.shape[0]
total_problem_cases

4656

In [35]:
messages = []
for m, expected, observed in [
    ("Problem cases G-K:", 4297, total_problem_cases),
]:
    result = "PASS" if expected==observed else "FAIL"
    messages.append([m,expected ,observed])
    
df_results = pd.DataFrame(messages, columns = ["Desription", "Expected", "Observed"])    
df_results["Result"] = df_results.Expected==df_results.Observed
df_results

,Desription,Expected,Observed,Result
0,Problem cases G-K:,4297,4656,False


## Summary

 Most of the counts for features  and cases with issues were accurate and alligned with the values from the Data Quality Comments tables. The values that didnt match were later discovered to be caused be the fact that the paper did not use all the integrity checks and that it instead likely only used the first 12 checks. Going back over the calculations and applying this knowledge led to identical results therefore when cleaning this dataset we will use the same logic and only run the first twelve integrity checks to find issues with cases and features.

In [36]:
messages = []
for m, expected, observed in [
    ("Identical Features:", 0, table_2_A(df)),
    ("Constant Features:", 0, table_2_B(df)),
    ("Features with missing values:", 0, table_2_C(df)),
    ("Features with conflicting values:", 2, table_2_D),
    ("Features with implausible values:", 1, total_implausible),
    ("Total features with issues:", 3, total_problem_features),
    ("Identical cases:", 4621, table_2_G),
    ("inconsistent  case values:", 100, table_2_H.shape[0]),
    ("Cases with missing values:", 0, table_2_I),
    ("Cases with conflicting values:", 129, table_2_J),
    ("Cases with implausible values:", 1084, table_2_K),
    ("Implausible cases :", 1084, table_2_K),
    ("Problem cases I-K:", 1163, total_data_quality_problem_cases),
    ("Problem cases G-K:", 4297, total_problem_cases),

]:
    result = "PASS" if expected==observed else "FAIL"
    messages.append([m,expected ,observed])
    
df_results = pd.DataFrame(messages, columns = ["Desription", "Expected", "Observed"])    
df_results["Result"] = df_results.Expected==df_results.Observed
df_results

,Desription,Expected,Observed,Result
0,Identical Features:,0,0,True
1,Constant Features:,0,0,True
2,Features with missing values:,0,0,True
3,Features with conflicting values:,2,2,True
4,Features with implausible values:,1,1,True
5,Total features with issues:,3,3,True
6,Identical cases:,4621,4621,True
7,inconsistent case values:,100,100,True
8,Cases with missing values:,0,0,True
9,Cases with conflicting values:,129,129,True


#### Comments
 - From the table above we can see that out findings match the ones from the Nasa Doc tables
 - We will now proceed to remove all the cases and features that are causing issues from the dataset

note: Our findings are matching because we used the same approach of only using specific checks

## Cleaning dataset by removing faulty features and cases - (NASA MDP Data Preprocessing Approach)

#### Step1: Remove cases with conflict feature values

In [37]:
failed_rows2 = pd.concat([check1, check3, check4, check5, check6, check7, check8, check9, check10, check11, check12])
df=df.drop(failed_rows2.index)

#### Step2: Remove cases with implausible values

In [38]:
df = df.drop(df[df['LOC_TOTAL'] == 0].index)

df = df.drop(df[(df < 0).any(axis=1)].index)

count_columns = [col for col in df.columns if 'count' in col]
df = df.drop(df[~df[count_columns].applymap(lambda x: isinstance(x, int)).all(axis=1)].index)

#### Step3: Remove identical cases

In [39]:
df=df.drop_duplicates(keep = False)

#### Step 4: remove inconsistent cases

In [40]:
grouped = df.groupby(df.columns[:-1].tolist())
inconsistent = grouped.filter(lambda x: len(x['defects'].unique()) > 1)
df=df.drop(inconsistent.index)

#### Step 5: Remove cases with missing values

In [41]:
df=df.dropna()

#### Step 6: Remove constant features

In [42]:
df.drop(columns=df.columns[df.apply(lambda x: x.nunique()) == 1], inplace=True)

#### Step 7: Remove identical features

In [48]:
df = df.loc[:, ~df.T.duplicated()]

##### Reseting datframe index

In [55]:
df = df.reset_index(drop=True)

## Saving cleaned dataset to new CSV

In [56]:
df.to_csv(f"{ROOT}/data/pc2.csv", index=False)
df.head(5)

,BRANCH_COUNT,CALL_PAIRS,LOC_CODE_AND_COMMENT,LOC_COMMENTS,CONDITION_COUNT,CYCLOMATIC_COMPLEXITY,CYCLOMATIC_DENSITY,DECISION_COUNT,DECISION_DENSITY,DESIGN_COMPLEXITY,DESIGN_DENSITY,EDGE_COUNT,ESSENTIAL_COMPLEXITY,ESSENTIAL_DENSITY,LOC_EXECUTABLE,PARAMETER_COUNT,HALSTEAD_CONTENT,HALSTEAD_DIFFICULTY,HALSTEAD_EFFORT,HALSTEAD_ERROR_EST,HALSTEAD_LENGTH,HALSTEAD_LEVEL,HALSTEAD_PROG_TIME,HALSTEAD_VOLUME,MAINTENANCE_SEVERITY,MODIFIED_CONDITION_COUNT,MULTIPLE_CONDITION_COUNT,NODE_COUNT,NORMALIZED_CYLOMATIC_COMPLEXITY,NUM_OPERANDS,NUM_OPERATORS,NUM_UNIQUE_OPERANDS,NUM_UNIQUE_OPERATORS,NUMBER_OF_LINES,PERCENT_COMMENTS,LOC_TOTAL,defects
0,1.0,4.0,7.0,24.0,0.0,1.0,0.13,0.0,0.0,1.0,1.00,6.0,1.0,0.0,1.0,0.0,17.88,7.43,986.77,0.04,34.0,0.13,54.82,132.83,1.0,0.0,0.0,7.0,0.03,13.0,21.0,7.0,8.0,34.0,96.88,8.0,False
1,1.0,1.0,11.0,3.0,0.0,1.0,0.08,0.0,0.0,1.0,1.00,2.0,1.0,0.0,1.0,0.0,42.62,7.81,2598.31,0.11,77.0,0.13,144.35,332.79,1.0,0.0,0.0,3.0,0.06,29.0,48.0,13.0,7.0,17.0,93.33,12.0,False
2,5.0,1.0,9.0,10.0,6.0,3.0,0.25,2.0,3.0,2.0,0.67,9.0,3.0,1.0,3.0,3.0,15.53,18.90,5545.97,0.10,64.0,0.05,308.11,293.44,1.0,2.0,3.0,8.0,0.13,27.0,37.0,10.0,14.0,24.0,86.36,12.0,False
3,1.0,0.0,3.0,5.0,0.0,1.0,0.25,0.0,0.0,1.0,1.00,1.0,1.0,0.0,1.0,1.0,20.84,3.94,323.05,0.03,21.0,0.25,17.95,82.04,1.0,0.0,0.0,2.0,0.09,9.0,12.0,8.0,7.0,11.0,88.89,4.0,False
4,3.0,0.0,1.0,0.0,4.0,2.0,1.00,2.0,2.0,1.0,0.50,7.0,1.0,0.0,1.0,3.0,11.42,6.67,507.65,0.03,20.0,0.15,28.20,76.15,0.5,1.0,2.0,7.0,0.50,10.0,10.0,6.0,8.0,4.0,50.00,2.0,False
